In [2]:
import os
import argparse
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as md
import numpy as np
import seaborn as sns
import glob
from scipy import stats
from dateutil import tz

In [3]:
N_STARLINK = 'Starlink'
N_SHAW = 'Shaw'

In [ ]:
src_dirs = 

In [4]:
df = pd.DataFrame()
for folder in args.src_dirs:
    for region in os.scandir(folder):
        pattern = f"{region.path}/*"
        print(pattern)
        df_temp = combine_csvs(glob.glob(pattern, recursive=True), 3)
        df_temp.index = pd.to_datetime(df_temp.index, unit='s')
        df_temp = df_temp.iloc[:,[2]]
        df_temp.rename(columns={df_temp.columns[0]: f"{folder}_{region.name}"}, inplace=True)

        df = df.merge(df_temp, how='outer', left_index=True, right_index=True)

print(df.shape[0])
num_regs = int(len(df.columns) / 2) # One for each of Shaw vs Starlink
print(num_regs)
cols = df.columns.tolist()

print(df.head())
new_cols = []
for col in cols:
    region = os.path.basename(col).replace('_',' ').strip()
    new_cols.append(region)

#regions = ['Mumbai', 'Sydney', 'Singapore', 
#        'N. California', 'London', 'Bahrain', 'Sao Paulo', 'Tokyo', 'Africa']

multi_cols = [
        [N_SHAW] * num_regs + [N_STARLINK] * num_regs,
        new_cols
        ]
df.set_axis(multi_cols, axis=1, inplace=True)

NameError: name 'args' is not defined

In [ ]:
# Filter
print(df.shape)
df = df.resample('H').mean()
df_regs_interp = df_regs_interp.interpolate(method='linear')
df_regs_interp = df_regs_interp.apply(lambda x: savgol_filter(x,41,1))
print(df)
print(df.shape)

#df = df.loc['2022-06-01 08':'2022-06-01 09']
print(df.shape)

df_unstacked = df.unstack().reset_index(name='latency')
print(df_unstacked)

In [ ]:
N_REGIONS = 'Regions'
df_unstacked.dropna(inplace=True)
df_unstacked.rename(columns={'level_0': 'type', 'level_1': N_REGIONS, 'level_2': 'timestamp'},
        inplace=True)
#print(df_unstacked.head())

#df_pivot = df_unstacked.pivot_table(index=[N_REGIONS, 'timestamp'], columns='type', values='latency')
#print(df_pivot.head())

g = sns.catplot(data=df_unstacked, x='type', y='latency', col=N_REGIONS, col_wrap=4, kind='box', height=2,
        aspect=0.895, showfliers=True)
g.set(yscale='log')
g.tight_layout()
(g.set_axis_labels('Provider', 'Latency (ms)')
        .set_titles("{col_name}"))

In [ ]:
#fig, ax = plt.subplots(figsize=(3.5,2))
fig, ax = plt.subplots(figsize=(20,10))
ax.xaxis.update_units(df.index)
sns.lineplot(data=df_unstacked, x="level_2", y="latency", hue="level_1", style="level_0")

for label in ax.get_xticklabels():
    label.set_rotation(45)
    label.set_ha('right')
ax.set_ylabel('Latency (ms)')

In [ ]:
########
#fig, axs = plt.subplots(1, num_regs, figsize=(7.16,5))
#ax_big = fig.add_subplot(111, frameon=False)

#df_regions = []
#for i in range(num_regs):
#    df_region = df.iloc[:,[i,(num_regs+i)]]
#    boxplt = sns.boxplot(x="variable", y="value", data=pd.melt(df_region), ax=axs[i], showfliers=False)

#    raw_region = df_region.columns[0]
#    region = raw_region.split('/')[1].replace('_',' ').strip()
#    boxplt.set(xlabel=region, ylabel=None)

#    #axs[i].set(yscale='log')
#    axs[i].set_xticklabels(labels=["Shaw", "Starlink"], fontsize=9)
#    axs[i].tick_params(labelsize=9)


#ax_big.set_xlabel("Network Provider and Region", fontsize=10, labelpad=30, fontweight='bold')
#ax_big.set_ylabel("Latency (ms)", fontsize=10, labelpad=30, fontweight='bold')
#ax_big.set_yticklabels([])
#ax_big.set_xticklabels([])
#ax_big.tick_params(
#    which='both',
#    bottom=False,
#    left=False,
#    right=False,
#    top=False)
#ax_big.grid(False)
#############

In [ ]:
plt.savefig(f'{args.filename}.pgf', bbox_inches='tight')
plt.show()